# Decoradores

## Tabla de contenidos
***
- [Decorando funciones](#id1)
<a href='#id1'></a>

- [La importancia de functools.wraps](#id2)
<a href='#id2'></a>

- [Concatenando o anidando decoradores](#id3)
<a href='#id3'></a>




***

Los decoradores son en esencia wrappers de funciones/classes que pueden ser utilizados para modificar el input, output, o incluso la clase/función en si misma antes de ejecutarla.

Esto puede ser logrado teniendo otra función que llama a la función interior, o mediante herencia de ciertas classes llamadas **mixins**.

Los decoradores brindan mucho "poder de reutilizar". Algunos decoradores muy útiles que ya vienen implementados en Python son 
`@property`, `@classmethod`, `@staticmethod`.

Es importante hacer notar que "envolver" una función crea una nueva función y lo hace más difícil alcanzar el interior de la función y sus propiedades. Un ejemplo de esto es la funcionalidad `help(funcion)` que ya viene implementada en Python.

<a id='id1'></a>

## Decorando funciones

Los decoradores son funciones o classes que "envuelven" otras funciones y/o clases.

In [9]:
def decorador(funcion):
    print(f"He ejecutado la función {funcion.__name__}")
    return funcion

def suma(a, b):
    return a + b

suma = decorador(suma)

He ejecutado la función suma


Para hacer la syntaxís más fácil de leer, Python permite decorar una función usando el operador `@` como atajo.

In [11]:
@decorador
def sumas(a,b):
    return a + b

He ejecutado la función sumas


Algunos usos de los decoradores incluye
- Registrar una función/clase
- Modificar el input de una función/clase
- Modificar el output de una función/clase
- Loggear instancias de clases o llamados de funciones

In [12]:
import functools

def decorador(funcion):
    #Este decorador asegura que imitamos la función que "envolvemos"
    @functools.wraps(funcion)
    def _decorador(a , b):
        
        result = funcion(a, b + 5)
        
        #Printeamos el nombre
        nombre = funcion.__name__
        print(f"{nombre}(a = {a}, b = {b}): {result}")
        
        #Retornamos un resultado modificado
        return result + 4
    
    return _decorador


@decorador
def funcion(a,b):
    return a + b

In [13]:
funcion(1,2)

funcion(a = 1, b = 2): 8


12

Como se puede observar, podemos modificar, añadir y/o remover argumentos. Podemos modificar el valor que retornamos o incluso llamar a una función completamente distinta si así lo deseamos. Además, de que podemos printear en consola todo el comportamiento si es necesario, lo cual puede ser bastante útil para debuggear.

<a id='id2'></a>

## La importancia de functools.wraps

Cuando sea que estés creando un decorador, siempre asegúrate de añadir `functools.wraps` para "envolver" la función interior. Sin "envolverla", vas a perder todas las propiedades originales de la función, lo que puede ocasionar un comportamiento confuso e inesperado.

In [14]:
import functools

def decorador(funcion):
    @functools.wraps(funcion)
    def _decorador(*args, **kwargs):
        return funcion(*args, **kwargs)
    
    return _decorador



@decorador
def suma(a, b):
    """Se suman a y b"""
    return a + b

In [15]:
help(suma)

Help on function suma in module __main__:

suma(a, b)
    Se suman a y b



In [17]:
suma.__name__

'suma'

El funcionamiento de `functools.wraps` es para nada mágico; copia y actualiza muchos atributos. Específicamente, los siguientes atributos son copiados:

- `__doc__`
- `__name__`
- `__module__`
- `__annotations__`
- `__qualname__`

<a id='id3'></a>

## Concatenando o anidando decoradores

Dado que estamos "envolviendo" funciones, no hay nada que nos impida el añadir múltiples decoradores. Los decoradores son inicializados partiendo desde adentro, pero son llamados partiendo por afuera.

In [18]:
import functools

def track(funcion = None, label = None):
    #Truco para añadir un argumento opcional a nuestro decorador
    if label and not funcion:
        return functools.partial(track, label = label)
    
    print(f"Inicializando {label}")
    
    @functools.wraps(funcion)
    def _track(*args, **kwargs):
        print(f"Llamando {label}")
        funcion(*args, **kwargs)
        print(f"Se llamó {label}")
    
    return _track

In [19]:
@track(label = 'exterior')
@track(label = "interior")
def funcion():
    print("Se generó un llamado de la función")

Inicializando interior
Inicializando exterior


In [20]:
funcion()

Llamando exterior
Llamando interior
Se generó un llamado de la función
Se llamó interior
Se llamó exterior


Como se puede observar los decoradores son llamados desde el exterior al interior antes de correr la función y al correrla son llamados desde el interior al exterior cuando se procesan los resultados.

## Registrando funciones usando decoradores

Esto puede ser muy útil en una interfaz de usuario (GUI).

In [23]:
import collections

class EventRegistry:
    def __init__(self):
        self.registry = collections.defaultdict(list)
    
    
    def on(self, *events):
        
        def _on(function):
            for event in events:
                self.registry[event].append(function)
            
            return function
        
        return _on
    
    
    def fire(self, event, *args, **kwargs):
        for function in self.registry[event]:
            function(*args, **kwargs)

In [24]:
events = EventRegistry()

In [27]:
@events.on('success', 'error')
def teardown(value):
    print(f'Tearing down got: {value}')

@events.on('success')
def success(value):
    print(f'Successfully executed: {value}')

In [29]:
events.fire('success', 'Everything is fine')

Tearing down got: Everything is fine
Tearing down got: Everything is fine
Successfully executed: Everything is fine


In [30]:
events.fire('error', 'Oops, some error here')

Tearing down got: Oops, some error here
Tearing down got: Oops, some error here


In [32]:
events.fire('non-existing', 'nothing to see here')

## Memorización usando decoradores

La memorización es un truco para recordar los resultados y así hacer correr el código de forma mucho más rápida en escenarios específicos. El truco es guardar un mapeo del input y del output esperado, así solo se tiene que calcular el valor una única vez. Un ejemplo de esto son los números de Fibonacci.

In [33]:
import functools